In [1]:
# Import libraries
import numpy as np
import pandas as pd


# Load bike rental and station data CSV files
bike_raw = pd.read_csv("seoul_bike_rental_data_2406.csv", encoding="cp949")
station_raw = pd.read_csv('seoul_bike_station_master_data.csv', encoding="cp949")

# Print dimensions of the two datasets
print(bike_raw.shape)
print()
print(station_raw.shape)

(5004330, 17)

(3340, 5)


In [3]:
# Create a copy of bike rental data for processing
bike_dat = bike_raw.copy()

# Rename and translate columns in bike rental data
bike_dat.rename(
    columns={
        '자전거번호': 'bike_id',
        '대여일시': 'rent_datetime',
        '대여 대여소번호': 'rent_station_number',
        '대여 대여소명': 'rent_station_name',
        '대여거치대': 'rent_station_hold',
        '반납일시': 'return_datetime',
        '반납대여소번호': 'return_station_number',
        '반납대여소명': 'return_station_name',
        '반납거치대': 'return_station_hold',
        '이용시간(분)': 'use_minutes',
        '이용거리(M)': 'use_meters',
        '생년': 'user_birth_year',
        '성별': 'user_sex',
        '이용자종류': 'user_type',
        '대여대여소ID': 'rent_station_id',
        '반납대여소ID': 'return_station_id',
        '자전거구분': 'bike_type',
    },
    inplace=True
)

# Replace invalid values with NA
bike_dat.replace('\\N', pd.NA, inplace=True)

# Add rental and return date and time variables 
bike_dat['rent_datetime'] = pd.to_datetime(bike_dat['rent_datetime'])
bike_dat['rent_date'] = bike_dat['rent_datetime'].dt.date
bike_dat['rent_hour'] = bike_dat['rent_datetime'].dt.hour
bike_dat['rent_weekday'] = bike_dat['rent_datetime'].dt.weekday
bike_dat['rent_is_weekend'] = bike_dat['rent_weekday'].apply(lambda x: True if x in [5, 6] else False)

bike_dat['return_datetime'] = pd.to_datetime(bike_dat['return_datetime'])
bike_dat['return_date'] = bike_dat['return_datetime'].dt.date
bike_dat['return_hour'] = bike_dat['return_datetime'].dt.hour
bike_dat['return_weekday'] = bike_dat['return_datetime'].dt.weekday
bike_dat['return_is_weekend'] = bike_dat['return_weekday'].apply(lambda x: True if x in [5, 6] else False)


# Fix rental station numbers to consistent format as return station numbers
bike_dat['rent_station_number'] = bike_dat['rent_station_number'].apply(lambda x: str(x).zfill(5))

In [5]:
# Assert that station numbers and IDs are uniquely matched
station_list = pd.concat([
    bike_dat[['rent_station_number', 'rent_station_id']].rename(columns={'rent_station_number': 'station_number','rent_station_id' : 'station_id'}), 
    bike_dat[['return_station_number', 'return_station_id']].rename(columns={'return_station_number': 'station_number','return_station_id' : 'station_id'})], ignore_index=True).drop_duplicates()

assert all(station_list['station_number'].value_counts() == 1)
assert all(station_list['station_id'].value_counts() == 1)

In [7]:
# Create a copy of station data for processing
station_info = station_raw.copy()

# Rename and translate columns in station data for consistency and readability
station_info.rename(
    columns={
        '대여소_ID': 'station_id',
        '주소1': 'address_1',
        '주소2': 'address_2',
        '위도': 'station_latitude',
        '경도': 'station_longitude',
    },
    inplace=True
)

# Replace invalid values with NA
station_info.replace('\\N', pd.NA, inplace=True)

# Assert that station IDs are unique in the station data
assert all(station_info['station_id'].value_counts() == 1)

In [9]:
# Merge station information into bike rental data for both rental and return stations
rent_station_info = station_info[['station_id', 'station_latitude', 'station_longitude']].rename(
    columns={
        'station_id': 'rent_station_id',
        'station_latitude': 'rent_station_latitude',
        'station_longitude': 'rent_station_longitude',
    })

return_station_info = station_info[['station_id', 'station_latitude', 'station_longitude']].rename(
    columns={
        'station_id': 'return_station_id',
        'station_latitude': 'return_station_latitude',
        'station_longitude': 'return_station_longitude',
    })

bike_dat = bike_dat.merge(rent_station_info, how='left', on='rent_station_id')
bike_dat = bike_dat.merge(return_station_info, how='left', on='return_station_id')

In [11]:
# Identify missing station information in the bike rental data
print(bike_dat['rent_station_latitude'].isnull().value_counts())
print()
print(bike_dat['return_station_latitude'].isnull().value_counts())

False    5004330
Name: rent_station_latitude, dtype: int64

False    4984352
True       19978
Name: return_station_latitude, dtype: int64


In [13]:
# Remove those with missing return information - not returned or software errors
bike_dat = bike_dat[bike_dat['return_station_id'].isnull() == False]

# Calculate numbers of unique bikes and rental stations
unique_stations = np.concatenate([bike_dat['rent_station_id'].drop_duplicates(), bike_dat['return_station_id'].drop_duplicates()])
unique_stations = np.unique(unique_stations)
unique_bikes = bike_dat['bike_id'].drop_duplicates()

num_unique_stations = len(unique_stations)
num_unique_bikes = len(unique_bikes)
num_rides = len(bike_dat)

print(f"Seould City's public bike rental system operated {num_rides:,} complete rides with {num_unique_bikes:,} bikes and {num_unique_stations:,} rental stations across the city in June 2024.")

Seould City's public bike rental system operated 4,984,352 complete rides with 39,197 bikes and 2,740 rental stations across the city in June 2024.


In [15]:
# Export and save processed bike rental data to a CSV file
bike_dat.to_csv('seoul_bike_rental_data_processed_2406.csv', index=False, encoding="cp949") 